# Train classifer on date phrases to find plan start date

In [85]:
import csv
import pandas as pd
import os
import classify
import extract_dates
import random

## Create training data

In [86]:
docs = pd.read_csv(os.path.join(data_path, 'text_narrowed.csv'))

### Randomly sample five plans

Do not overwrite labelled document set unless you want to relabel. relabel_docs = False uses previously randomly sample plans.

In [87]:
relabel = False
if relabel:
    docs_to_label = docs[docs.p_innovation >.5].sample(n = 5)
    docs_to_label.index
    docs_to_label.head(5)

In [88]:
if not relabel:
    docs_to_label = docs[docs.title == "Pearland ISD"]
    docs_to_label = docs_to_label.append(docs[docs.title == "Ector County ISD"])
    docs_to_label = docs_to_label.append(docs[docs.title == "Dimmitt ISD"])
    docs_to_label = docs_to_label.append(docs[docs.title == "Sunnyvale ISD"])
    docs_to_label = docs_to_label.append(docs[docs.title == "Sunray ISD"])
    docs_to_label.head(5)

## Collect phrases surrounding date match. Currently, function collects 8 words before and 6 words after.

In [89]:
date_phrases = []
for text in docs_to_label.text:
    phrases = extract_dates.get_phrase_list(text = text)
    for phrase in phrases:
        date_phrases.append(phrase)
date_phrases = [str(item) for item in date_phrases]
date_phrases

['the District Educational Improvement Committee Authored: April 2016 - September 2016 INTRODUCTION In April',
 'Improvement Committee Authored: April 2016 - September 2016 INTRODUCTION In April 2016, the',
 'April 2016 - September 2016 INTRODUCTION In April 2016, the Pearland ISD Board ofTrustees',
 'newly permitted by the Texas Legislature under HB 1842 (see TAC Chapter 102)',
 'It is hoped that the legislature, in 2017, will expand the areas for',
 'of the Texas Education Agency. • In 2016, all STAAR scores/subjects',
 ', as recorded by the Houston Chronicle in 2016, rated Pearland ISD as the',
 'in the Greater Houston area. • In 2016, 19 of 22 campuses received',
 'plan. That planning process began in late 2015 and will be finalized in late',
 'late 2015 and will be finalized in late 2016. Within that plan will be',
 'of Innovation legislation (TAC 102) of 2015. The following items (and',
 ', with the legislature meeting again in January 2017, we hope for greater flexibility',
 'gr

### Save random sample to dataframe. Do not change seed unless you want to relabel.

In [90]:
phrase_df = pd.DataFrame({'text':date_phrases})
phrase_df = phrase_df.sample(50, random_state = 414)
phrase_df.head()

,text
29,"102.1305 adopted to be effective September 13,..."
108,"of Innovation. Process On December 12, 2016, t..."
24,"102.1301 adopted to be effective September 13,..."
64,term of the Plan will begin with the 2017-2018...
75,"for public review and comment March 11, 2017 –..."


In [91]:
i = 1
for item in phrase_df.text:
    print(i, '-', item)
    i = i + 1

1 - 102.1305 adopted to be effective September 13, 2016, 41 TexReg 7089. §
2 - of Innovation. Process On December 12, 2016, the Board appointed a District
3 - 102.1301 adopted to be effective September 13, 2016, 41 TexReg 7089. §
4 - term of the Plan will begin with the 2017-2018 school year and terminate
5 - for public review and comment March 11, 2017 – April 9, 2017 April
6 - certain mandates. On November, 28, 2016, the Sunnyvale ISD Board of
7 - effective September 13, 2016, 41 TexReg 7089. §102.1303. Eligibility.
8 - of Innovation Meeting/Approval February 23, 2017 Posted to SISD Website February 24
9 - , with the legislature meeting again in January 2017, we hope for greater flexibility
10 - plan by 2/3 majority vote February 9, 2017 Public meeting February 9, 2017
11 - to TEA Commissioner, Publish April 28, 2017 TERM The term of this Plan
12 - 2017 Posted to SISD Website February 24, 2017 Public Approval by District of Innovation
13 - and the community. On February 28, 2017, the

In [92]:
cats = [False, False, False, True, False, 
        False, False, False, False, False,
        False, False, False, False, False,
        False, True, False, False, False,
        False, False, False, True, False,

        False, False, False, False, False,
        False, False, False, False, False,
        False, False, True, False, False, 
        False, False, False, True, False,
        False, False, False, False, False]

In [93]:
phrase_df['TERM'] = cats

## Add extra true examples to beginning of dataset

In [94]:
true_phrases = ["2018-2023",
'This plan will be in effect for the 2017-2018 school year through the 2021-2022 school year.',
'Effective Dec. 13, 2016 – Dec. 13, 2021',
'The District of Innovation Plan will become effective in September 2018 and will remain in effect for five years, through June 2023.',
'Yorktown Independent School District Final District of Innovation Plan 2017-2018.',
'the plan will begin in the fall of 2018 and continue until the spring of 2023',
'Wortham ISD District of Innovation Plan 2017-2018 through 2021-2022',
'The District of Innovation Plan will become effective in April 2017 and will remain in effect for five years, through April 2022.',
'Effective 2018-2019 to 2022-2023',
'The term of the plan is for five years, beginning February 14, 2017, and ending February 14, 2022, unless terminated.',
'Approved by the Board of Trustees September 25, 2017 2017-2022 The Lueders-Avoca Independent School District (L-A ISD)',
'The five year plan will take effect August 2017 and end August 2022 unless rescinded or terminated by the Board of Trustees.',
'CLYDE CISD DISTRICT OF INNOVATION PLAN 2018 - 2023 Clyde Consolidated Independent School District',
'The term of this Plan is for five years, beginning April 1, 2017, and ending December 31, 2021, unless terminated or amended earlier by the Board in accordance with TEC §12A.006 and §12A.008.',
'Therefore, this plan will begin with the 2017-2018 school year and conclude at the end of the 2021-2022 school year unless terminated by the Board of Trustees.',
'The five year plan will take effect August 2017 and end August 2022 unless rescinded or terminated by the Board of Trustees.']

In [95]:
true_phrases = [
    'remain in effect for the next five years (2019-2023). The plan may be amended at',
    'District of Innovation Plan 2016-2021 District of Innovation Plan 2016-2021 House',
    'years. Therefore, this plan will begin with the 2017-2018 school year and conclude at the',
    'years, beginning at the start of the 2017‐2018 school year  and concluding at the',
    'remain in effect for the next five years (2019-2023). The plan may be amended', 
    'is for five years, beginning in August 2017 through the 2021-2022 school year,',
    'A DISTRICT PLAN FOR INNOVATION AND LOCAL CONTROL 2017-2022 APPROVED BY THE BOARD OF TRUSTEES',
    'years, beginning at the start of the 2017-2018 school year and ending at the',
    'AISD District Plan of Innovation June 1, 2017 – June 1, 2022 House Bill (HB) 1842 passed during',
    'District of Innovation June 2017 - May 2021 School Start Date',
    'District of Innovation Plan will become effective in June, 2017, and will remain in effect',
    'will commence with the adoption at the February 27, 2017, Board of Trustees regular meeting and conclude',
    'ISD’s Innovation Plan will begin with the 2017-2018 school year and conclude at the',
    'of the Plan is for five years, beginning March 1, 2017 and ending March 1, 2021,',
    'INDEPENDENT SCHOOL DISTRICT District of Innovation Plan 2018-2023 SCUC – LEADING WITH PASSION AND',
    'years, beginning at the start of the 2017-18 school year and ending at the',
    'the plan will begin in the fall of 2018 and continue until the spring of',
    'for five years, beginning April 1, 2017, and ending December 31, 2021',
    'CLYDE CISD DISTRICT OF INNOVATION PLAN 2018 - 2023 Clyde Consolidated Independent School',
    'Effective September 2016 – September 2021 2 What is',
    'Effective Dec. 13, 2016 – Dec. 13, 2021 2',
    'of Innovation Plan will become effective in April 2017 for the 2017-2018 school',
    'AISD District of Innovation 2016-2017 AISD Mission: Educate every student for',
    'District of Innovation Plan 2017-2018 Committe members include Jack Franty,'

]
phrase_df2 = pd.DataFrame({'text':true_phrases})
phrase_df2['TERM'] = True
phrase_df = phrase_df2.append(phrase_df)

In [96]:
phrase_df = phrase_df.sample(frac = 1)

In [97]:
print('The first n-10 of documents are used for training. Training dataset statistics:')
split = int(len(phrase_df) - 10)
print(phrase_df.head(split)['TERM'].value_counts())

print('The last 10 of documents are used for testing. Testing dataset statistics:')
print(phrase_df.tail(10)['TERM'].value_counts())

The first n-10 of documents are used for training. Training dataset statistics:
False    40
True     24
Name: TERM, dtype: int64
The last 10 of documents are used for testing. Testing dataset statistics:
True     5
False    5
Name: TERM, dtype: int64


## Train classifier

In [98]:
texts, cats = classify.reformat_cat_from_df(df = phrase_df,
                                      text_col = 'text',
                                      label_col = 'TERM',
                                      category = 'TERM')

In [99]:
output_dir = os.path.join(data_path, 'date_term_classifier')

In [100]:
classify.train_classifier_and_evaluate(texts=texts, 
                                        cats=cats,
                                        n_test = 10,
                                        model=None, 
                                        output_dir=output_dir, 
                                        n_iter=10, 
                                        categories=['TERM'])

Created blank 'en' model
Using 74 examples (64 training, 10 evaluation)
Training the model...
LOSS 	  P  	  R  	  F  
6.190	0.500	0.400	0.444
3.550	0.667	0.400	0.500
3.073	0.600	0.600	0.600
0.949	0.600	0.600	0.600
1.929	0.600	0.600	0.600
1.698	0.667	0.800	0.727
0.518	0.667	0.800	0.727
1.262	0.667	0.800	0.727
0.677	0.667	0.800	0.727
0.425	0.667	0.800	0.727
No text {'TERM': 4.539787187241018e-05}
Saved model to /Users/kylieleblancKylie/dofis/code/exemptions/../../data/date_term_classifier
Loading from /Users/kylieleblancKylie/dofis/code/exemptions/../../data/date_term_classifier
No text {'TERM': 4.539787187241018e-05}
